<center style="font-size:48px;">Clean Up</center>

In [147]:
import pandas as pd 

cars = pd.read_csv('../Data/car-assignments.csv')
cc = pd.read_csv('../Data/cc_data.csv', encoding='cp1252', parse_dates=['timestamp'])
gps = pd.read_csv('../Data/gps.csv', parse_dates=['Timestamp'])
loyalty = pd.read_csv('../Data/loyalty_data.csv', encoding='cp1252', parse_dates=['timestamp'])

In [148]:
locations = gps.merge(cars, left_on='id', right_on='CarID', how='left')
locations.drop(columns='CarID', inplace =True)
locations.head()

,Timestamp,id,lat,long,LastName,FirstName,CurrentEmploymentType,CurrentEmploymentTitle
0,2014-01-06 06:28:01,35,36.076225,24.874689,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
1,2014-01-06 06:28:01,35,36.076220,24.874596,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
2,2014-01-06 06:28:03,35,36.076211,24.874443,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
3,2014-01-06 06:28:05,35,36.076217,24.874253,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
4,2014-01-06 06:28:06,35,36.076214,24.874167,Vasco-Pais,Willem,Executive,Environmental Safety Advisor


In [149]:
print("People with first name Driver : {}".format(sum(sum([locations.FirstName == "Truck"]))))
print("People with last name Driver : {}".format(sum(sum([locations.LastName == "Driver"]))))

People with first name Driver : 0
People with last name Driver : 0


In [150]:
locations['CurrentEmploymentType'] = locations['CurrentEmploymentType'].fillna('Facilities')
locations['CurrentEmploymentTitle'] = locations['CurrentEmploymentTitle'].fillna('Truck Driver')
locations['LastName'] = locations['LastName'].fillna('Driver')
locations['FirstName'] = locations['FirstName'].fillna('Truck')
locations['LastName'] = locations.apply(lambda x: 'Driver_{}'.format(x['id']) if x['LastName'] == 'Driver' else x['LastName'], axis =1 )

In [151]:
timeUnit = ['day', 'hour', 'minute', 'second']
for unit in timeUnit:
    if unit == 'day':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.day)
    if unit == 'hour':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.hour)
    if unit == 'minute':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.minute)
    if unit == 'second':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.second)

In [152]:
locations[['lat', 'long']] = locations.groupby(['day', 'hour', 'minute', 'second', 'FirstName', 'LastName'])['lat', 'long'].transform('mean')
locations.drop_duplicates()

/tmp/ipykernel_42806/470201690.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  locations[['lat', 'long']] = locations.groupby(['day', 'hour', 'minute', 'second', 'FirstName', 'LastName'])['lat', 'long'].transform('mean')


,Timestamp,id,lat,long,LastName,FirstName,CurrentEmploymentType,CurrentEmploymentTitle,day,hour,minute,second
0,2014-01-06 06:28:01,35,36.076223,24.874643,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,1
2,2014-01-06 06:28:03,35,36.076211,24.874443,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,3
3,2014-01-06 06:28:05,35,36.076217,24.874253,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,5
4,2014-01-06 06:28:06,35,36.076214,24.874167,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,6
5,2014-01-06 06:28:07,35,36.076191,24.874056,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,7
...,...,...,...,...,...,...,...,...,...,...,...,...
685164,2014-01-19 20:56:43,30,36.058110,24.902130,Resumir,Felix,Security,Security Group Manager,19,20,56,43
685165,2014-01-19 20:56:47,30,36.058258,24.901774,Resumir,Felix,Security,Security Group Manager,19,20,56,47
685166,2014-01-19 20:56:48,30,36.058296,24.901711,Resumir,Felix,Security,Security Group Manager,19,20,56,48
685167,2014-01-19 20:56:49,30,36.058304,24.901620,Resumir,Felix,Security,Security Group Manager,19,20,56,49


In [153]:
cc['Is_Loyalty'] = False
loyalty['Is_Loyalty'] = True

In [154]:
timeUnit = ['day', 'hour', 'minute', 'second']
for unit in timeUnit:
    if unit == 'day':
        cc[unit] = cc['timestamp'].apply(lambda x: x.day)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.day)
    if unit == 'hour':
        cc[unit] = cc['timestamp'].apply(lambda x: x.hour)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.hour)
    if unit == 'minute':
        cc[unit] = cc['timestamp'].apply(lambda x: x.minute)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.minute)
    if unit == 'second':
        cc[unit] = cc['timestamp'].apply(lambda x: x.second)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.second)

cc['cents'] = round(cc.apply(lambda x: (x['price'] % 1) * 100,  axis = 1))
loyalty['cents'] = round(loyalty.apply(lambda x: (x['price'] % 1) * 100,  axis = 1))

In [155]:
for index, row in cc.iterrows():
    first = row['FirstName']
    last = row['LastName']
    location = row['location']
    day = row['day']
    cents = row['cents']
    temp = loyalty[loyalty.FirstName == first]
    temp = temp[temp.LastName == last]
    temp = temp[temp.location == location]
    temp = temp[temp.day == day]
    temp = temp[temp.cents == cents]
    if len(temp) >= 1:
        cc.loc[index, 'Is_Loyalty'] = True
        cc.loc[index, 'price'] = temp.price.values

buys = pd.concat([cc, loyalty]).drop_duplicates(['FirstName', 'LastName', 'location', 'day', 'cents'], keep='first')
buys = buys.merge(cars, left_on=['LastName', 'FirstName'], right_on=['LastName', 'FirstName'], how= 'left')


In [162]:
buys[buys.CurrentEmploymentType.isnull()]

,timestamp,location,price,FirstName,LastName,Is_Loyalty,day,hour,minute,second,cents,CarID,CurrentEmploymentType,CurrentEmploymentTitle
5,2014-01-06 07:38:00,Brew've Been Served,4.17,Cornelia,Lais,True,6,7,38,0,17.0,NaN,NaN,NaN
7,2014-01-06 07:43:00,Brew've Been Served,9.60,Mat,Bramar,True,6,7,43,0,60.0,NaN,NaN,NaN
9,2014-01-06 07:47:00,Hallowed Grounds,16.53,Emile,Arpa,True,6,7,47,0,53.0,NaN,NaN,NaN
15,2014-01-06 07:54:00,Hallowed Grounds,12.93,Rachel,Pantanal,True,6,7,54,0,93.0,NaN,NaN,NaN
19,2014-01-06 08:00:00,Brew've Been Served,4.27,Ruscella,Mies Haber,True,6,8,0,0,27.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,2014-01-15 00:00:00,Katerina’s Café,38.81,Ruscella,Mies Haber,True,15,0,0,0,81.0,NaN,NaN,NaN
1501,2014-01-16 00:00:00,Ouzeri Elian,31.21,Rachel,Pantanal,True,16,0,0,0,21.0,NaN,NaN,NaN
1502,2014-01-16 00:00:00,Guy's Gyros,36.48,Emile,Arpa,True,16,0,0,0,48.0,NaN,NaN,NaN
1503,2014-01-17 00:00:00,Hippokampos,23.54,Anda,Ribera,True,17,0,0,0,54.0,NaN,NaN,NaN


In [ ]:
pd.concat([cc, loyalty]).drop_duplicates(['FirstName', 'LastName', 'location', 'day', 'cents'], keep='first')

,timestamp,location,price,FirstName,LastName,Is_Loyalty,day,hour,minute,second,cents
0,2014-01-06 07:28:00,Brew've Been Served,11.34,Edvard,Vann,True,6,7,28,0,34.0
1,2014-01-06 07:34:00,Hallowed Grounds,12.22,Hideki,Cocinaro,True,6,7,34,0,22.0
2,2014-01-06 07:35:00,Brew've Been Served,8.33,Stenig,Fusil,True,6,7,35,0,33.0
3,2014-01-06 07:36:00,Hallowed Grounds,16.72,Birgitta,Frente,True,6,7,36,0,72.0
4,2014-01-06 07:37:00,Brew've Been Served,4.24,Sven,Flecha,True,6,7,37,0,24.0
...,...,...,...,...,...,...,...,...,...,...,...
1337,2014-01-18 00:00:00,Ouzeri Elian,53.36,Bertrand,Ovan,True,18,0,0,0,36.0
1341,2014-01-18 00:00:00,Katerina’s Café,14.22,Sten,Sanjorge Jr.,True,18,0,0,0,22.0
1357,2014-01-19 00:00:00,Hippokampos,11.56,Axel,Calzas,True,19,0,0,0,56.0
1377,2014-01-19 00:00:00,Guy's Gyros,23.21,Elsa,Orilla,True,19,0,0,0,21.0


In [ ]:
buys[['location', 'price', 'FirstName', 'LastName']].duplicated().sum()

848

In [ ]:
sven = buys[buys.FirstName == 'Birgitta'].sort_values(['location', 'day'])
sven['cents'] = round(sven.apply(lambda x: (x['price'] % 1) * 100,  axis = 1))
sven.drop_duplicates(['location', 'cents', 'day'], keep=False).sort_values(['location', 'day'])

,timestamp,location,price,FirstName,LastName,Is_Loyalty,CarID,CurrentEmploymentType,CurrentEmploymentTitle,day,hour,minute,second,cents
178,2014-01-06 21:03:00,Katerina’s Café,10.72,Birgitta,Frente,False,18.0,Engineering,Geologist,6,21,3,0,72.0
216,2014-01-09 00:00:00,Katerina’s Café,26.00,Birgitta,Frente,True,18.0,Engineering,Geologist,9,0,0,0,0.0
189,2014-01-11 15:15:00,Shoppers' Delight,157.45,Birgitta,Frente,False,18.0,Engineering,Geologist,11,15,15,0,45.0


In [ ]:
buys.sort_values(['FirstName', 'LastName', 'location', 'day'])

,timestamp,location,price,FirstName,LastName,Is_Loyalty,CarID,CurrentEmploymentType,CurrentEmploymentTitle,day,hour,minute,second
1662,2014-01-17 13:11:00,Abila Zacharo,68.78,Ada,Campo-Corrente,False,10.0,Executive,SVP/CIO,17,13,11,0
1694,2014-01-17 00:00:00,Abila Zacharo,8.78,Ada,Campo-Corrente,True,10.0,Executive,SVP/CIO,17,0,0,0
1642,2014-01-09 21:27:00,Albert's Fine Clothing,229.98,Ada,Campo-Corrente,False,10.0,Executive,SVP/CIO,9,21,27,0
1674,2014-01-09 00:00:00,Albert's Fine Clothing,229.98,Ada,Campo-Corrente,True,10.0,Executive,SVP/CIO,9,0,0,0
1631,2014-01-06 12:00:00,Brewed Awakenings,4.35,Ada,Campo-Corrente,False,10.0,Executive,SVP/CIO,6,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,2014-01-16 00:00:00,Jack's Magical Beans,6.95,Willem,Vasco-Pais,True,35.0,Executive,Environmental Safety Advisor,16,0,0,0
1963,2014-01-17 13:37:00,Kalami Kafenion,105.32,Willem,Vasco-Pais,False,35.0,Executive,Environmental Safety Advisor,17,13,37,0
1996,2014-01-17 00:00:00,Kalami Kafenion,25.32,Willem,Vasco-Pais,True,35.0,Executive,Environmental Safety Advisor,17,0,0,0
1965,2014-01-18 13:28:00,Kalami Kafenion,38.91,Willem,Vasco-Pais,False,35.0,Executive,Environmental Safety Advisor,18,13,28,0


In [ ]:
buys[buys.FirstName == 'Sven'].sort_values(['location', 'day']).head(50)

,timestamp,location,price,FirstName,LastName,Is_Loyalty,CarID,CurrentEmploymentType,CurrentEmploymentTitle,day,hour,minute,second
260,2014-01-15 13:47:00,Abila Zacharo,20.44,Sven,Flecha,False,17.0,Information Technology,IT Technician,15,13,47,0
267,2014-01-18 14:55:00,Ahaggo Museum,120.20,Sven,Flecha,False,17.0,Information Technology,IT Technician,18,14,55,0
299,2014-01-18 00:00:00,Ahaggo Museum,120.20,Sven,Flecha,True,17.0,Information Technology,IT Technician,18,0,0,0
237,2014-01-06 07:37:00,Brew've Been Served,4.24,Sven,Flecha,False,17.0,Information Technology,IT Technician,6,7,37,0
270,2014-01-06 00:00:00,Brew've Been Served,4.24,Sven,Flecha,True,17.0,Information Technology,IT Technician,6,0,0,0
240,2014-01-07 07:27:00,Brew've Been Served,5.08,Sven,Flecha,False,17.0,Information Technology,IT Technician,7,7,27,0
273,2014-01-07 00:00:00,Brew've Been Served,5.08,Sven,Flecha,True,17.0,Information Technology,IT Technician,7,0,0,0
242,2014-01-08 08:01:00,Brew've Been Served,3.42,Sven,Flecha,False,17.0,Information Technology,IT Technician,8,8,1,0
276,2014-01-08 00:00:00,Brew've Been Served,3.42,Sven,Flecha,True,17.0,Information Technology,IT Technician,8,0,0,0
245,2014-01-09 07:48:00,Brew've Been Served,5.30,Sven,Flecha,False,17.0,Information Technology,IT Technician,9,7,48,0


<div>
    <span  style=width:700px;display:inline-block;align:left">
        <a href="./FurtherEDA.ipynb"><< Further Exploratory Data Analysis</a>
    </span>
    <span style="width:700px;display:inline-block;" align="right">
        <a href="./Model.ipynb">Modeling >></a>
    </span>
</div>
<div>
    <center>
        <span style="width:250px;display:inline-block">
            <a href="../Master.ipynb">Master Notebook</a>
        </span>
        <span style="width:250px;display:inline-block">
            <a href="../README.md">Table of Contents</a>
        </span>
    <center>
</div>